In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
plt.style.use("seaborn-v0_8-deep")
plt.rcParams.update({
    "text.usetex": True,  # Use LaTeX for text rendering
    "font.family": "serif",  # Use serif font
    "font.serif": ["Computer Modern Roman"],  # Use default LaTeX font
})

In [2]:
df_lstm = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/lstm_hyper_opt.csv')
df_rnn = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/rnn_hyper_opt.csv')
df_vqc = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/vqc_hyper_opt.csv')
df_mlp = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/mlp_hyper_opt.csv')
df_qrnn = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/qrnn_paper_hyper_opt.csv')
df_qlstm_paper = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/qlstm_paper_hyper_opt.csv')
df_qlstm_le = pd.read_csv('/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Results/qlstm_linear_enhanced_paper_hyper_opt.csv')

In [3]:
#df_vqc_reuploading_expencnorm = df_vqc[df_vqc['Ansatz'].str.startswith('reuploading_expencnorm')]
df_vqc_reuploading_expencnorm = df_vqc[df_vqc['Ansatz'].str.startswith('ruexp_')]
df_vqc_plain = df_vqc[df_vqc['Ansatz'].str.startswith('paper_rivera-ruiz_no_inputlayer_')]
df_vqc_dressed = df_vqc[df_vqc['Ansatz'].str.startswith('paper_rivera-ruiz_with_inputlayer_')]
df_qrnn_no_reset = df_qrnn[df_qrnn['Ansatz']== 'paper_no_reset']

In [ ]:
sequence_lengths = [4, 8, 16]
prediction_steps_mackey = [1, 70, 140]
prediction_steps_henon = [1, 2, 4]
prediction_steps_lorenz = [1, 13, 25]

models = {'d-QNN': (df_vqc_dressed, '#D65A31'), 'ru-QNN': (df_vqc_reuploading_expencnorm, '#E27D60'), 'QRNN': (df_qrnn_no_reset, '#FFBF46'), 'QLSTM': (df_qlstm_paper, '#C5A880'), 'le-QLSTM': (df_qlstm_le, '#6B4226'), 'MLP': (df_mlp, '#006D77'),'RNN': (df_rnn, '#00A8CC'), 'LSTM': (df_lstm, '#4A90E2')}
data = {'mackey_1000': (prediction_steps_mackey, 'Mackey-Glass'), 'henon_1000': (prediction_steps_henon, 'Hénon'), 'lorenz_1000': (prediction_steps_lorenz, 'Lorenz')}

In [ ]:
for data_label, (prediction_steps, data_name) in data.items():
    fig, axs = plt.subplots(3, 3, figsize=(3*2.4, 3*2.7), sharex=False)
    handles_list = []
    labels_list = []
    for i, sequence_length in enumerate(sequence_lengths):
        for j, prediction_step in enumerate(prediction_steps):
            ax = axs[j, i]
            for model, (df, color) in models.items():
                df_filtered = df[(df['Sequence Length'] == sequence_length) & (df['Data'] == data_label) & (df['Prediction Step'] == prediction_step)]
                df_filtered = df_filtered.sort_values('Num Parameters')
                ax.scatter(df_filtered['Num Parameters'], df_filtered['MSE Testing Median'], marker='x', label=model, color=color)
            handles, labels = ax.get_legend_handles_labels()
            handles_list.extend(handles)
            labels_list.extend(labels)
            ax.set_title(f'Seq: {sequence_length}, Pred: {prediction_step}')
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.grid(True)
    unique_handles_labels = list(dict(zip(labels_list, handles_list)).items())
    unique_handles = [h for l, h in unique_handles_labels]
    unique_labels = [l for l, h in unique_handles_labels]
    fig.legend(unique_handles, unique_labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, 0.002))
    fig.text(0.54, 0.11, 'Number of Parameters', ha='center')
    fig.text(0.02, 0.52, 'Median MSE', va='center', rotation='vertical')
    fig.suptitle(f'{data_name}')

    plt.tight_layout()  # Adjust layout to accommodate the legend outside
    fig.subplots_adjust(left=0.13, right=0.95, top=0.92, bottom=0.18, wspace=0.5, hspace=0.35)
    plt.savefig(f'/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Plots/All_models/all_models_mse_over_parameters_{data_label}_new.pdf')

In [ ]:
models_specific = {'d-QNN': (df_vqc_dressed, '#D65A31'), 'QRNN': (df_qrnn_no_reset, '#FFBF46'), 'le-QLSTM': (df_qlstm_le, '#6B4226'), 'LSTM': (df_lstm, '#4A90E2')}
pred_specific = [1, 25]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(4, 4), sharex=False)
handles_list = []
labels_list = []
for j, prediction_step in enumerate(pred_specific):
    ax = axs[j]
    for model, (df, color) in models_specific.items():
        df_filtered = df[(df['Sequence Length'] == 16) & (df['Data'] == "lorenz_1000") & (df['Prediction Step'] == prediction_step)]
        df_filtered = df_filtered.sort_values('Num Parameters')
        ax.errorbar(df_filtered['Num Parameters'], df_filtered['MSE Testing Median'], yerr=df_filtered['MSE Testing Mad'], marker='o', linestyle='none', label=model, color=color, capsize=7, markeredgecolor='black', markeredgewidth=0.5)
    handles, labels = ax.get_legend_handles_labels()
    handles_list.extend(handles)
    labels_list.extend(labels)
    ax.set_title(f'Pred: {prediction_step}', fontsize=11)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=11)  # Adjust fontsize for x ticks
    ax.tick_params(axis='y', labelsize=11)  # Adjust fontsize for y ticks
unique_handles_labels = list(dict(zip(labels_list, handles_list)).items())
unique_handles = [h for l, h in unique_handles_labels]
unique_labels = [l for l, h in unique_handles_labels]
fig.legend(unique_handles, unique_labels, loc='upper right', ncol=1, bbox_to_anchor=(0.95, 0.87), fontsize=9)
fig.text(0.54, 0.02, 'Number of Parameters', ha='center', fontsize=11)
fig.text(0.02, 0.52, 'Median MSE', va='center', rotation='vertical', fontsize=11)
fig.suptitle(f'{data_name}, Seq: 16', fontsize=11)

plt.tight_layout()  # Adjust layout to accommodate the legend outside
fig.subplots_adjust(left=0.18, right=0.95, top=0.87, bottom=0.12, wspace=0.5, hspace=0.4)
plt.savefig(f'/tikhome/tfellner/Projects/VQA_timeseries_benchmark/Plots/All_models/specific_models_mse_over_parameters_new.pdf')